In [1]:
import pandas as pd
import subprocess
import logging

from tqdm import tqdm

In [2]:
download_desc_file = "../data/download_descriptions.json"

df = pd.read_json(download_desc_file)
for i, row in tqdm(df.iterrows(), total=len(df)):
    if row["has-en-sub"]:
        command = f"youtube-dl -o ../data/00_raw/{i} --write-sub --sub-lang=en --skip-download {row.url}".split()
    else:
        command = f"youtube-dl -o ../data/00_raw/{i} --write-auto-sub --skip-download {row.url}".split()
    
    response = subprocess.run(command, capture_output=True)
    if response.returncode != 0:
        logging.error(f"Error while downloading the {i}-th video. \n {response.stderr}")

100%|███████████████████████████████████████████| 11/11 [00:18<00:00,  1.69s/it]
